In [18]:
!pip3 install voyageai
!pip3 install scikit-learn


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.7 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 159.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 123.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 143.7 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [1]:
from anthropic import Anthropic
import voyageai


In [2]:
# Load environment variables
from helper import load_env, get_voyage_client
load_env()

In [3]:
# Simple document
documents = [
    "The Mediterranean diet emphasizes fish, olive oil, and vegetables, believed to reduce chronic diseases.",
    "Photosynthesis in plants converts light energy into glucose and produces essential oxygen.",
    "20th-century innovations, from radios to smartphones, centered on electronic advancements.",
    "Rivers provide water, irrigation, and habitat for aquatic species, vital for ecosystems.",
	  "Apple’s conference call to discuss fourth fiscal quarter results and business updates is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET.",
    "Shakespeare's works, like 'Hamlet' and 'A Midsummer Night's Dream,' endure in literature."
]

In [4]:
# embed the document using Voyage embedding model.
vo = get_voyage_client()
documents_embeddings = vo.embed(documents, model="voyage-3.5", input_type="document").embeddings

In [5]:
vo.embed(documents, model="voyage-3.5", input_type="document").embeddings

[[0.0478527657687664,
  0.015432325191795826,
  0.002147634979337454,
  0.025793416425585747,
  0.037925511598587036,
  0.012564346194267273,
  -0.02115057408809662,
  -0.02702796831727028,
  -0.02791471965610981,
  0.003821927821263671,
  0.08311575651168823,
  0.005344569217413664,
  -0.048121556639671326,
  0.019854500889778137,
  0.0002106770989485085,
  -0.020648173987865448,
  0.015314953401684761,
  0.0503227673470974,
  -0.016123609617352486,
  -0.025918733328580856,
  0.00835761521011591,
  0.00012792731286026537,
  -0.09673710912466049,
  0.03671411797404289,
  -0.00197884370572865,
  -0.02248796634376049,
  0.03562985733151436,
  -0.017508449032902718,
  -0.07537882030010223,
  0.003932034131139517,
  0.0021932385861873627,
  0.007574840914458036,
  0.011864209547638893,
  0.013269024901092052,
  -0.021272486075758934,
  -0.01748756319284439,
  0.020047016441822052,
  -0.0696687400341034,
  -0.018295763060450554,
  0.015521488152444363,
  0.0332515686750412,
  -0.00227998942

In [6]:
len(vo.embed(documents, model="voyage-3.5", input_type="document").embeddings)

6

In [7]:
vo

In [8]:
dir(vo)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_params',
 'api_key',
 'contextualized_embed',
 'count_tokens',
 'count_usage',
 'embed',
 'multimodal_embed',
 'rerank',
 'retry_controller',
 'tokenize',
 'tokenizer']

In [9]:
# user query/user request.
query = "When is Apple's conference call scheduled?"

In [12]:
query_embeddings = vo.embed([query], model="voyage-3.5", input_type="query").embeddings[0]

In [13]:
len(vo.embed([query], model="voyage-3.5", input_type="query").embeddings)

1

In [14]:
# implementing Nearest neighbor search.
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def k_nearest_neighbors(query_embedding, documents_embeddings, k=5):
  query_embedding = np.array(query_embedding) # convert to numpy array
  documents_embeddings = np.array(documents_embeddings) # convert to numpy array

  # Reshape the query vector embedding to a matrix of shape (1, n) to make it compatible with cosine_similarity
  query_embedding = query_embedding.reshape(1, -1)

  # Calculate the similarity for each item in data
  cosine_sim = cosine_similarity(query_embedding, documents_embeddings)

  # Sort the data by similarity in descending order and take the top k items
  sorted_indices = np.argsort(cosine_sim[0])[::-1]

  # Take the top k related embeddings
  top_k_related_indices = sorted_indices[:k]
  top_k_related_embeddings = documents_embeddings[sorted_indices[:k]]
  top_k_related_embeddings = [list(row[:]) for row in top_k_related_embeddings] # convert to list

  return top_k_related_embeddings, top_k_related_indices

In [15]:
def cosine_similarity_using_dot_product(documents, doc_embds, query_embd):
    similarities = np.dot(doc_embds, query_embd)
    retrieved_id = np.argmax(similarities)
    return documents[retrieved_id]

In [17]:
# Use the nearest neighbor algorithm to find the document with the highest similarity
retrieved_embd, retrieved_embd_index = k_nearest_neighbors(query_embeddings, documents_embeddings, k=1)
retrieved_doc = [documents[index] for index in retrieved_embd_index]

print(retrieved_doc)

['Apple’s conference call to discuss fourth fiscal quarter results and business updates is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET.']


In [18]:
# Use the k-nearest neighbor algorithm to identify the top-k documents with the highest similarity
retrieved_embds, retrieved_embd_indices = k_nearest_neighbors(query_embeddings, documents_embeddings, k=3)
retrieved_docs = [documents[index] for index in retrieved_embd_indices]

print(retrieved_docs)

['Apple’s conference call to discuss fourth fiscal quarter results and business updates is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET.', '20th-century innovations, from radios to smartphones, centered on electronic advancements.', 'Photosynthesis in plants converts light energy into glucose and produces essential oxygen.']


In [19]:
# Refinement with rerankers
# reranker rerank the documents for semantic relevance against the query.
# Reranking
documents_reranked = vo.rerank(
  query,
  retrieved_docs,
  model="rerank-2.5",
  top_k=3
)

In [20]:
for r in documents_reranked.results:
  print(f"Document: {r.document}")
  print(f"Relevance Score: {r.relevance_score}")
  print(f"Index: {r.index}")
  print("\n")

Document: Apple’s conference call to discuss fourth fiscal quarter results and business updates is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET.
Relevance Score: 0.94140625
Index: 0


Document: 20th-century innovations, from radios to smartphones, centered on electronic advancements.
Relevance Score: 0.28515625
Index: 1


Document: Photosynthesis in plants converts light energy into glucose and produces essential oxygen.
Relevance Score: 0.255859375
Index: 2




In [21]:
# simple RAG Bot with Anthropic'S Claude LLM.
# Take the retrieved document and use it as a prompt for the text generation model
prompt = f"Based on the information: '{retrieved_doc}', generate a response of {query}"


In [22]:
client = Anthropic()
MODEL_NAME="claude-3-5-sonnet-20241022"

In [23]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=1000,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.content[0].text)

Apple's conference call is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET.
